Die Vorhersage des West-Nil-Virus (WNV) in Chicago ist entscheidend, um Ressourcen effizient zu nutzen und die Ausbreitung des Virus zu verhindern. In diesem Projekt erstellen wir ein prädiktives Modell zur Vorhersage des Virusaufkommens.

## 2. CRISP-DM: Project Understanding

### 2.1 Content

### 2.1 Content

### 2.2 Project Goal

### 2.3 Domain Knowledge

In [1]:
# import pandas as pd
# import numpy as np

# import matplotlib.pyplot as plt
# #%matplotlib inline

# from sklearn.neighbors import KernelDensity
# # Source: https://www.kaggle.com/code/vascovv/west-nile-heatmap
# mapdata = np.loadtxt("../data/mapdata_copyright_openstreetmap_contributors.txt")
# traps = pd.read_csv('../data/train.csv')[['Date', 'Trap','Longitude', 'Latitude', 'WnvPresent']]

# alpha_cm = plt.cm.Reds
# alpha_cm._init()
# alpha_cm._lut[:-3,-1] = abs(np.logspace(0, 1, alpha_cm.N) / 10 - 1)[::-1]
# aspect = mapdata.shape[0] * 1.0 / mapdata.shape[1]
# lon_lat_box = (-88, -87.5, 41.6, 42.1)

# sigthings = traps[traps['WnvPresent'] > 0]
# sigthings = sigthings.groupby(['Date', 'Trap','Longitude', 'Latitude']).max()['WnvPresent'].reset_index()
# X = sigthings[['Longitude', 'Latitude']].values
# kd = KernelDensity(bandwidth=0.02)
# kd.fit(X)

# xv,yv = np.meshgrid(np.linspace(-88, -87.5, 100), np.linspace(41.6, 42.1, 100))
# gridpoints = np.array([xv.ravel(),yv.ravel()]).T
# zv = np.exp(kd.score_samples(gridpoints).reshape(100,100))
# plt.figure(figsize=(10,14))
# plt.imshow(mapdata, 
#            cmap=plt.get_cmap('gray'), 
#            extent=lon_lat_box, 
#            aspect=aspect)
# plt.imshow(zv, 
#            origin='lower', 
#            cmap=alpha_cm, 
#            extent=lon_lat_box, 
#            aspect=aspect)

# locations = traps[['Longitude', 'Latitude']].drop_duplicates().values
# plt.scatter(locations[:,0], locations[:,1], marker='x')

# plt.savefig('heatmap.png')